In [39]:
import requests, pandas as pd
from pathlib import Path
from sqlalchemy import create_engine

url = "https://github.com/lerocha/chinook-database/raw/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
p = Path("db/Chinook.sqlite")
p.write_bytes(requests.get(url, timeout=60).content)

engine = create_engine(f"sqlite:///{p.as_posix()}")

# Consulta
query = f'''
SELECT 
  t.Name AS TrackName, 
  ar.Name AS ArtistName, 
  a.Title AS AlbumTitle, 
  COALESCE(t.Composer, 'unknown') AS composer, 
  g.Name AS Genre,
  COUNT(il.TrackId) AS quantity, 
  COALESCE(SUM(il.UnitPrice), 0) AS total_income,
  COUNT(pt.TrackId) AS popularity

FROM Track t
LEFT JOIN Genre g   ON t.GenreId = g.GenreId 
LEFT JOIN Album a   ON t.AlbumId = a.AlbumId
LEFT JOIN Artist ar ON a.ArtistId = ar.ArtistId
LEFT JOIN InvoiceLine il ON t.TrackId = il.TrackId
LEFT JOIN Invoice i ON i.InvoiceId = il.InvoiceId
LEFT JOIN PlaylistTrack pt ON pt.TrackId = t.TrackId

GROUP BY t.Name, ar.Name, a.Title, t.Composer, g.Name
ORDER BY total_income DESC

'''

display(pd.read_sql(query, engine))



,TrackName,ArtistName,AlbumTitle,composer,Genre,quantity,total_income,popularity
0,"Scheherazade, Op. 35: I. The Sea and Sindbad's...",Chicago Symphony Orchestra & Fritz Reiner,Scheherazade,Nikolai Rimsky-Korsakov,Classical,10,9.90,10
1,"Suite No. 3 in D, BWV 1068: III. Gavotte I & II","Academy of St. Martin in the Fields, Sir Nevil...",Bach: Orchestral Suites Nos. 1 - 4,Johann Sebastian Bach,Classical,10,9.90,10
2,"Symphonie Fantastique, Op. 14: V. Songe d'une ...",Michael Tilson Thomas & San Francisco Symphony,Berlioz: Symphonie Fantastique,Hector Berlioz,Classical,10,9.90,10
3,Gay Witch Hunt,The Office,"The Office, Season 3",unknown,TV Shows,4,7.96,4
4,Hot Girl,The Office,"The Office, Season 1",unknown,TV Shows,4,7.96,4
...,...,...,...,...,...,...,...,...
3493,É Preciso Saber Viver,Titãs,Volume Dois,Erasmo Carlos/Roberto Carlos,Alternative & Punk,0,0.00,3
3494,É Uma Partida De Futebol,Skank,O Samba Poconé,Samuel Rosa,Rock,0,0.00,3
3495,É que Nessa Encarnação Eu Nasci Manga,Various Artists,Axé Bahia 2001,Lucina/Luli,Pop,0,0.00,2
3496,Óia Eu Aqui De Novo,Gilberto Gil,As Canções de Eu Tu Eles,unknown,Soundtrack,0,0.00,2
